In [1]:
from gurobipy import *  # import the optimize solver Gurobi

In [2]:
number_of_chemical = 4 # Set the index i for number types of chemical(1, 2, 3, 4)
number_of_ingredient = 3 # Set the index j for number types of ingredient(A, B, C)
m = Model() # Import and create the model

Using license file C:\Users\chaoz\gurobi_licence\gurobi.lic
Academic license - for non-commercial use only


In [3]:
# Set the input Parameter: 
cost_chem = [8, 10, 11, 14] # Cost for each kind of chemical per pound
ingredient = [[0.03, 0.02, 0.01], [0.06, 0.04, 0.01], [0.1, 0.03, 0.04], [0.12, 0.09, 0.04]]   # the amount of each active ingredient in one pound of each chemical
ingred_lim = [0.07, 0.05, 0.03]  # Weight limitation for each active ingredient
total_amount = 5000 # the company must produce 5000 pounds of the drug
chem_3_lim = 600  #  At least 600 pounds of chemical 3 must be used

In [4]:
# Set the Variable list: Number of pound chemical i needed to produce drug 
# Set the variable to continuous number
nx = [] 
for i in range(number_of_chemical):
    nx.append(m.addVar(vtype=GRB.CONTINUOUS, name='nx{}'.format(i + 1)))


In [5]:
# Caculate total pounds of ingredient j can be extracted from chemical i
t = []
for i in range(number_of_chemical):
    s = []
    for j in range(number_of_ingredient):
        s.append(nx[i] * ingredient[i][j] )
    t.append(s)

In [7]:
# Caculate total pounds of ingredient j in the drug
t1 = []
for j in range(number_of_ingredient):
    t1.append(sum([t[i][j] for i in range(number_of_chemical)]))

In [13]:
m.update()
print(t1)

[<gurobi.LinExpr: 0.03 nx1 + 0.06 nx2 + 0.1 nx3 + 0.12 nx4>, <gurobi.LinExpr: 0.02 nx1 + 0.04 nx2 + 0.03 nx3 + 0.09 nx4>, <gurobi.LinExpr: 0.01 nx1 + 0.01 nx2 + 0.04 nx3 + 0.04 nx4>]


In [8]:
# Set the Minimize Obijective: Total cost
m.setObjective(quicksum([cost_chem[i] * nx[i] for i in range(number_of_chemical)]) ,  GRB.MINIMIZE)

In [9]:
# Set Non Negative number of order
c1=[]
for i in range(number_of_chemical):
    c1.append(m.addConstr(nx[i] >= 0))

In [10]:
# Satisfy minimum total amount of drug 5000 pounds
c2 = m.addConstr( sum(nx) >= total_amount)

In [11]:
# Satisfy at least 600 pounds of chemical 3 must be used
c3 = m.addConstr( nx[2] >= chem_3_lim)

In [14]:
# Satisfy weight limitation for each active ingredient
c4=[]
for j in range(number_of_ingredient):
    c4.append(m.addConstr(t1[j] >= (sum([nx[i] for i in range(number_of_chemical)]) * ingred_lim[j])))

In [15]:
# Run the optimize solver
m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 9 rows, 4 columns and 21 nonzeros
Model fingerprint: 0xd60b6e10
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [8e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+02, 5e+03]
Presolve removed 5 rows and 0 columns
Presolve time: 0.42s
Presolved: 4 rows, 4 columns, 16 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.3598194e+04   1.092169e+03   0.000000e+00      0s
       2    5.5833333e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.44 seconds
Optimal objective  5.583333333e+04


In [17]:
# Get the Optimal Solution for X
m.printAttr('X')


    Variable            X 
-------------------------
         nx1      1666.67 
         nx3      1388.89 
         nx4      1944.44 


In [18]:
# Get the objective values
m.ObjVal

55833.33333333333